In [1]:
## First Part: Load the data: 
import pandas as pd 
import numpy as np

def shuffle(x_,y_):
    s = np.arange(x_.shape[0])
    s = np.random.shuffle(s)

    x_re = x_[s]
    y_re = y_[s]

    x_re = np.reshape(x_re,(len(x_),48,48))
    y_re = np.reshape(y_re,(len(y_)))
    return x_re,y_re

def read_fer(path):
    # train_path = "C:\\Users\\Jiaming Nie\\Documents\\Work-DeepGlint\Facial\datasets\\train.csv"
    data = pd.read_csv(path, dtype='a')
    label = np.array(data['emotion'])
    img_data = np.array(data['pixels'])

    N_sample = label.size

    x_data = np.zeros((N_sample, 48 * 48))
    # train_label = np.zeros((N_sample, 7), dtype=int)
    y_label = np.zeros(N_sample, dtype=int)
    # print(train_label)

    for i in range(N_sample):
        x = img_data[i]
        x = np.fromstring(x, dtype=float, sep=' ')
        x_max = x.max()
        x = x / (x_max + 0.0001)
        # print x_max
        # print x
        x_data[i] = x
        y_label[i] = int(label[i])
        # train_label[i, label[i]] = 1 #This step seems direct one-hot encoding
        # print(y_label[i])
        #    img_x = np.reshape(x, (48, 48))
        #    plt.subplot(10,10,i+1)
        #    plt.axis('off')
        #    plt.imshow(img_x, plt.cm.gray)

    x_data = np.reshape(x_data,(len(x_data),48,48))
    return x_data, y_label

def ReadData_fer():
    # ubuntu path
    #path_train = "/home/jiaming/code/DeepGlint-Work/Facial/datasets/train.csv"
    #path_test = "/home/jiaming/code/DeepGlint-Work/Facial/datasets/test.csv"

    # windows path
    path_train = "/train/trainset/1/train.csv"
    path_test = "/train/trainset/1/test.csv"
    path_vali = "/train/trainset/1/val.csv"
    
    x_train, y_train = read_fer(path_train)
    x_test, y_test = read_fer(path_test)
    x_vali, y_vali = read_fer(path_vali)

    x_train, y_train = shuffle(x_train, y_train)
    x_test, y_test = shuffle(x_test, y_test)
    x_vali, y_vali = shuffle(x_vali, y_vali)

    return x_train,y_train,x_test,y_test,x_vali,y_vali

def zca_whitening(X):
    """
        Function to compute ZCA whitening matrix (aka Mahalanobis whitening).
        INPUT:  X: [M x N] matrix.
            Rows: Variables
            Columns: Observations
        OUTPUT: ZCAMatrix: [M x M] matrix
        """
    mean_ = np.mean(X)
    X = X - mean_
    # Covariance matrix [column-wise variables]: Sigma = (X-mu)' * (X-mu) / N
    sigma = np.cov(X, rowvar=True)  # [M x M]
    # Singular Value Decomposition. X = U * np.diag(S) * V
    U, S, V = np.linalg.svd(sigma)
    # U: [M x M] eigenvectors of sigma.
    # S: [M x 1] eigenvalues of sigma.
    # V: [M x M] transpose of U
    # Whitening constant: prevents division by zero
    epsilon = 0.1
    # ZCA Whitening matrix: U * Lambda * U'
    ZCAMatrix = np.dot(U, np.dot(np.diag(1.0 / np.sqrt(S + epsilon)), U.T))  # [M x M]
    return ZCAMatrix

def normalization(x_):

    length = len(x_)

    for i in range(length):
        x_[i] = zca_whitening(x_[i])

    return x_


In [2]:
x_train,y_train,x_test,y_test,x_vali,y_vali = ReadData_fer()

# Normalization
x_train = normalization(x_train)
x_test = normalization(x_test)
x_vali = normalization(x_vali)

x_train = x_train.reshape((len(x_train), 48, 48, 1))
x_test = x_test.reshape((len(x_test), 48, 48, 1))
x_vali = x_vali.reshape((len(x_vali),48,48,1))

print(x_train.shape)
print(x_test.shape)

(28709, 48, 48, 1)
(3589, 48, 48, 1)


In [22]:
from scipy.misc import imread, imresize
import keras
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, Dropout
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import SGD
import sys
from keras.layers import Input, Dense, Conv2D, MaxPooling2D
from keras.models import Model
from keras.layers.merge import concatenate
from keras.regularizers import l2
from keras.metrics import top_k_categorical_accuracy
from keras.utils import to_categorical
from keras.layers import Flatten
import time
import matplotlib.pyplot as plt
from keras import optimizers

import tensorflow as tf
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# 設定 Keras 使用的 Session
tf.keras.backend.set_session(sess)

In [25]:

def inception_model(input, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
    conv_1x1 = Conv2D(filters=filters_1x1, kernel_size=(1, 1), padding='same', activation='relu', kernel_regularizer=l2(0.01))(input)

    conv_3x3_reduce = Conv2D(filters=filters_3x3_reduce, kernel_size=(1, 1), padding='same', activation='relu', kernel_regularizer=l2(0.01))(input)

    conv_3x3 = Conv2D(filters=filters_3x3, kernel_size=(3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.01))(conv_3x3_reduce)

    conv_5x5_reduce  = Conv2D(filters=filters_5x5_reduce, kernel_size=(1, 1), padding='same', activation='relu', kernel_regularizer=l2(0.01))(input)

    conv_5x5 = Conv2D(filters=filters_5x5, kernel_size=(5, 5), padding='same', activation='relu', kernel_regularizer=l2(0.01))(conv_5x5_reduce)

    maxpool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(input)

    maxpool_proj = Conv2D(filters=filters_pool_proj, kernel_size=(1, 1), strides=(1, 1), padding='same', activation='relu', kernel_regularizer=l2(0.01))(maxpool)

    inception_output = concatenate([conv_1x1, conv_3x3, conv_5x5, maxpool_proj], axis=3)  # use tf as backend

    return inception_output

def define_model(weight_path = None):
    input = Input(shape=(48, 48, 1))

    conv1_7x7_s2 = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu', kernel_regularizer=l2(0.01))(input)

    maxpool1_3x3_s2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(conv1_7x7_s2)

    conv2_3x3_reduce = Conv2D(filters=64, kernel_size=(1, 1), padding='same', activation='relu', kernel_regularizer=l2(0.01))(maxpool1_3x3_s2)

    conv2_3x3 = Conv2D(filters=192, kernel_size=(3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.01))(conv2_3x3_reduce)

    maxpool2_3x3_s2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(conv2_3x3)

    inception_3a = inception_model(input=maxpool2_3x3_s2, filters_1x1=64, filters_3x3_reduce=96, filters_3x3=128, filters_5x5_reduce=16, filters_5x5=32, filters_pool_proj=32)

    inception_3b = inception_model(input=inception_3a, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=192, filters_5x5_reduce=32, filters_5x5=96, filters_pool_proj=64)

    maxpool3_3x3_s2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(inception_3b)

    inception_4a = inception_model(input=maxpool3_3x3_s2, filters_1x1=192, filters_3x3_reduce=96, filters_3x3=208, filters_5x5_reduce=16, filters_5x5=48, filters_pool_proj=64)

    inception_4b = inception_model(input=inception_4a, filters_1x1=160, filters_3x3_reduce=112, filters_3x3=224, filters_5x5_reduce=24, filters_5x5=64, filters_pool_proj=64)

    inception_4c = inception_model(input=inception_4b, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=256, filters_5x5_reduce=24, filters_5x5=64, filters_pool_proj=64)

    inception_4d = inception_model(input=inception_4c, filters_1x1=112, filters_3x3_reduce=144, filters_3x3=288, filters_5x5_reduce=32, filters_5x5=64, filters_pool_proj=64)

    inception_4e = inception_model(input=inception_4d, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320, filters_5x5_reduce=32, filters_5x5=128, filters_pool_proj=128)

    maxpool4_3x3_s2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(inception_4e)

    inception_5a = inception_model(input=maxpool4_3x3_s2, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320, filters_5x5_reduce=32, filters_5x5=128, filters_pool_proj=128)

    inception_5b = inception_model(input=inception_5a, filters_1x1=384, filters_3x3_reduce=192, filters_3x3=384, filters_5x5_reduce=48, filters_5x5=128, filters_pool_proj=128)

    averagepool1_7x7_s1 = AveragePooling2D(pool_size=(7, 7), strides=(7, 7), padding='same')(inception_5b)

    drop1 = Dropout(rate=0.4)(averagepool1_7x7_s1)
    #print(drop1)

    #flatten = keras.layers.core.Flatten(drop1)
    #print("flatten ",flatten)

    #linear = Dense(units=7, activation='softmax', kernel_regularizer=l2(0.01))(keras.layers.core.Flatten(drop1))
    drop1 = Flatten()(drop1)
    linear = Dense(units=7, activation='softmax', kernel_regularizer=l2(0.01))(drop1)
    last = linear

    model = Model(inputs=input, outputs=last)
    #model.summary()
    sgd = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    batch_size = 16
    EPOCH = 100
    
    results = []
    start_time = time.time()

    history = model.fit(x_train, to_categorical(y_train),
              batch_size=batch_size,
              epochs=EPOCH,
              validation_data=(x_vali, to_categorical(y_vali)))

    average_time_per_epoch = (time.time() - start_time) / EPOCH
    results.append((history, average_time_per_epoch))
    plt.style.use('ggplot')
    ax1 = plt.subplot2grid((2, 2), (0, 0))
    ax1.set_title('Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epochs')
    ax2 = plt.subplot2grid((2, 2), (1, 0))
    ax2.set_title('Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epochs')
    ax3 = plt.subplot2grid((2, 2), (0, 1), rowspan=2)
    ax3.set_title('Time')
    ax3.set_ylabel('Seconds')

    for result in results:
        ax1.plot(result[0].epoch, result[0].history['val_acc'], label='Test')
        ax1.plot(result[0].epoch, result[0].history['acc'], label='Train')
        ax2.plot(result[0].epoch, result[0].history['val_loss'], label='Test')
        ax2.plot(result[0].epoch, result[0].history['loss'], label='Train')

    ax1.legend()
    ax2.legend()
    ax3.bar(np.arange(len(results)), [x[1] for x in results],
            align='center')
    plt.tight_layout()
    plt.show()
    
    score = model.evaluate(x_test, to_categorical(y_test), verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    #return model


In [26]:
if __name__ == '__main__':
    model = define_model()

Train on 28709 samples, validate on 3589 samples
Epoch 1/100
28709/28709 [==============================] - 186s 6ms/step - loss: 81.2457 - acc: 0.2492 - val_loss: 78.3993 - val_acc: 0.2494
Epoch 2/100
28709/28709 [==============================] - 176s 6ms/step - loss: 75.7095 - acc: 0.2513 - val_loss: 73.0855 - val_acc: 0.2494
Epoch 3/100
28709/28709 [==============================] - 180s 6ms/step - loss: 70.5964 - acc: 0.2513 - val_loss: 68.1647 - val_acc: 0.2494
Epoch 4/100
28709/28709 [==============================] - 180s 6ms/step - loss: 65.8541 - acc: 0.2513 - val_loss: 63.5958 - val_acc: 0.2494
Epoch 5/100
28709/28709 [==============================] - 179s 6ms/step - loss: 61.4489 - acc: 0.2513 - val_loss: 59.3500 - val_acc: 0.2494
Epoch 6/100
28709/28709 [==============================] - 179s 6ms/step - loss: 57.3541 - acc: 0.2513 - val_loss: 55.4031 - val_acc: 0.2494
Epoch 7/100
28709/28709 [==============================] - 181s 6ms/step - loss: 53.5474 - acc: 0.2513 - 

28709/28709 [==============================] - 183s 6ms/step - loss: 3.5410 - acc: 0.2513 - val_loss: 3.4866 - val_acc: 0.2494
Epoch 58/100
28709/28709 [==============================] - 179s 6ms/step - loss: 3.4321 - acc: 0.2513 - val_loss: 3.3812 - val_acc: 0.2494
Epoch 59/100
28709/28709 [==============================] - 184s 6ms/step - loss: 3.3300 - acc: 0.2513 - val_loss: 3.2826 - val_acc: 0.2494
Epoch 60/100
28709/28709 [==============================] - 183s 6ms/step - loss: 3.2347 - acc: 0.2513 - val_loss: 3.1904 - val_acc: 0.2494
Epoch 61/100
28709/28709 [==============================] - 184s 6ms/step - loss: 3.1453 - acc: 0.2513 - val_loss: 3.1040 - val_acc: 0.2494
Epoch 62/100
28709/28709 [==============================] - 185s 6ms/step - loss: 3.0620 - acc: 0.2513 - val_loss: 3.0233 - val_acc: 0.2494
Epoch 63/100
28709/28709 [==============================] - 185s 6ms/step - loss: 2.9836 - acc: 0.2513 - val_loss: 2.9477 - val_acc: 0.2494
Epoch 64/100
28709/28709 [=======

KeyboardInterrupt: 